In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from src.Utils import normalize_data, unnormalize_data, plot_image, plot_value_array

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic

def load_data(path=None):
    if path is None:
        cifar = keras.datasets.cifar10
        return cifar.load_data()
    else:
        d=[]
        l=[]
        for i in range(5):
            batch = unpickle(path + "/data_batch_" + str(i+1))
            d.append(batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1))
            l.append(np.asarray(batch[b'labels']).reshape((10000,1)))
        train_images = np.concatenate( [ d[i] for i in range(5) ])
        train_labels = np.concatenate( [ l[i] for i in range(5) ])
        batch = unpickle(path + "/test_batch")
        test_images = batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
        test_labels = np.asarray(batch[b'labels']).reshape((10000,1))
        return (train_images, train_labels), (test_images, test_labels)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()
#(train_images, train_labels), (test_images, test_labels) = load_data("data/cifar-10-batches-py")

In [ ]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [ ]:
train_images, mu_X, sigma_X = normalize_data(train_images)
test_images, mu_X, sigma_X = normalize_data(test_images, mu_X, sigma_X)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(1024),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.Dense(128),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=0, momentum=0.9, nesterov=False)
keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=10e-7, decay=0.0)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=1024, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)


In [ ]:
predictions = model.predict(test_images)

In [ ]:
i = 522
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, unnormalize_data(test_images, mu_X, sigma_X)/255., class_names)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)